In [1]:
from dotenv import load_dotenv
from os import environ
from databases import Database
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
load_dotenv('.env')

DATABASE_URL = environ.get('DB_CONNECTION')+'://'+environ.get('DB_USERNAME')+':'+environ.get('DB_PASSWORD')+'@'+environ.get('DB_HOST')+':'+environ.get('DB_PORT')+'/'+environ.get('DB_DATABASE')
database = Database(DATABASE_URL)

In [3]:
hotels_df = pd.read_csv('../archive/Hotel_details.csv')
rooms_df = pd.read_csv('../archive/hotels_RoomPrice.csv')

In [4]:
cities = hotels_df['city'].to_list()
cities = list(set(cities))
cities = pd.DataFrame(cities, columns=["name"])
cities = cities.reset_index()
cities.head()

,index,name
0,0,Dun Luain
1,1,Stary Dvur
2,2,San Leone
3,3,Thorpe on the Hill
4,4,Longueau


In [5]:
await database.connect()
query = "INSERT INTO cities(id, name, image) VALUES"
image_link = 'https://unsplash.com/photos/Nyvq2juw4_o/download?ixid=MnwxMjA3fDB8MXxzZWFyY2h8Mnx8Y2l0eXxlbnwwfHx8fDE2NjUwMDAzNjM&force=true&w=640'
for index, city in cities.iterrows():
    query = query + "("+str(index)+", '"+city['name'].replace("'", "''")+"','"+image_link+"'),"
query = query[:-1]
print("The query was created")
await database.execute(query=query)

The query was created


In [6]:
hotels = hotels_df[['id', 'hotelname', 'starrating', 'address', 'city']]
hotels = hotels.dropna()
hotels = hotels.reset_index()

In [29]:
load_in_each_query = 2000
for x in range(0, hotels['index'].count(), load_in_each_query):
    query = "INSERT INTO hotels(id, name, address, image, credit, star, city_id) VALUES"
    for index, hotel in hotels.iterrows():
        if x<=index<x+load_in_each_query:
            query = query + "("+str(hotel['id'])+", '"+hotel['hotelname'].replace("'", "''")+"', '"+hotel['address'].replace("'", "''")+"', '"+image_link+"', 0, "+str(hotel["starrating"])+", "+str(cities.loc[cities['name']==hotel['city']].index[0])+"),"
    query = query[:-1]
    await database.execute(query=query)

102955
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The query was created
The